In [1]:
# import packages
# pandas is used to manipulate and clean DataFrames (tables)
import pandas as pd
# in this case, numpy is used to assign values, similar to CASE/WHEN in SQL
import numpy as np
# os is used to navigate file paths
import os
# glob is used to loop through files in multiple folders
import glob

In [ ]:
### Pre-work Step 1
# This step specifies the column widths that you see in the "caltpa file layout.txt" file.
# Specify the column widths for use in pd.read_fwf() below.
# Need to start the index at 0.
col_widths = [
    (0,9), #1
    (9,17), #2
    (17,20), #3
    (20,21), #4
    (21,38), #5
    (38,48), #6
    (48,49), #7
    (49,50), #8
    (50,58), #9
    (58,59), #10
    (59,61), #11 
    (61,63), #12
    (63,65),
    (65,68),
    (68,71),
    (71,74),
    (74,77),
    (77,80),
    (80,83),
    (83,86),
    (86,89),
    (89,92),
    (92,93),
    (93,96),
    (96,98),
    (98,-1)
    ] 

In [ ]:
### Pre-work Step 2
# Specify the column names for use in pd.read_fwf() below.
col_names = [
    'SSN', 
    'Reporting Date', 
    'Field Code', 
    'Blank_1', 
    'Last Name', 
    'First Name', 
    'Middle Initial', 
    'Blank_2', 
    'Birthdate',
    'Blank_3', 
    'Number of Attempts', 
    'Cycle Passing Status',
    'Cycle Total Score',
    'Rubric 1',
    'Rubric 2',
    'Rubric 3',
    'Rubric 4',
    'Rubric 5',
    'Rubric 6',
    'Rubric 7',
    'Rubric 8',
    'Rubric 9',
    'Blank_4',
    'Institution',
    'Preparation Program',
    'Field Specialty'
    ]

In [ ]:
# Make an empty list. This is where each file name will go.
file_list = []

In [ ]:
# Change the file path here to the folder where the data is stored
path = 'Data/'

In [ ]:
# Using glob to look at every file in a folder, then append it to a list
# Reference: https://stackoverflow.com/questions/18262293/how-to-open-every-file-in-a-folder
for filename in glob.glob(os.path.join(path, '*.asc')):
    with open(os.path.join(os.getcwd(), filename), 'r') as f: # open in readonly mode
      # Read in the data using fixed widths.
      # Reference: https://towardsdatascience.com/parsing-fixed-width-text-files-with-pandas-f1db8f737276
      current_file = pd.read_fwf(
          str(filename),
          colspecs = col_widths,
          names = col_names,
          header = None,
          )
      file_list.append(current_file)

In [ ]:
# Combine the files within the file_list to one text file
# Reference: https://stackoverflow.com/questions/51960263/pandas-python-merge-multiple-file-text
f_combined = pd.concat(file_list, axis=0)

In [ ]:
# Convert the text file to a DataFrame.
# By convention, name the new DataFrame "df"
df = pd.DataFrame(f_combined)

In [ ]:
# New method: drop by column name instead.
df.drop(
    ['Blank_1', 'Blank_2', 'Blank_3', 'Blank_4'],
    axis = 1,
    inplace = True
    )


In [ ]:
# Change column types to string
df['Field Code'] = df['Field Code'].astype(str)

In [ ]:
# Pad the zeros in "Field Code".
df['Field Code'] = df['Field Code'].str.zfill(3)

In [ ]:
# Insert the campus code in the last column based on "Institution" using np.select().
# Reference: https://stackoverflow.com/questions/49228596/pandas-case-when-default-in-pandas
# 308 == 55 == Campus A
# 326 == 63 == Campus B
# 334 == 65 == Campus C
df['campusID'] = np.select(
    [
        df['Institution'] == 101,
        df['Institution'] == 102,
        df['Institution'] == 103,
    ],
    [
        11,
        12,
        13
    ],
    default = 0
)

In [ ]:
# Print the unique values of 'campusID' to see if the np.select worked correctly
print(df['campusID'].unique())

In [ ]:
# Print the head of the DataFrame for inspection.
print(df.head())

In [ ]:
# Export to .csv with index = False.
# Optional: For testing, remove the ID column, birthdate column, and Last Name column, then assign to a new DataFrame.
df_new = df.drop(['ID_Num', 'Birthdate', 'Last Name'], axis=1)

In [ ]:
# Save the DataFrame to the Data/ folder.
save_location = path + "CALTPA_processed_file.csv"
df_new.to_csv(save_location, index = False)